In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s3e16/sample_submission.csv
/kaggle/input/playground-series-s3e16/train.csv
/kaggle/input/playground-series-s3e16/test.csv


In [2]:
train = pd.read_csv('/kaggle/input/playground-series-s3e16/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s3e16/test.csv')

In [3]:
train.head()

,id,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Age
0,0,I,1.5250,1.1750,0.3750,28.973189,12.728926,6.647958,8.348928,9
1,1,I,1.1000,0.8250,0.2750,10.418441,4.521745,2.324659,3.401940,8
2,2,M,1.3875,1.1125,0.3750,24.777463,11.339800,5.556502,6.662133,9
3,3,F,1.7000,1.4125,0.5000,50.660556,20.354941,10.991839,14.996885,11
4,4,I,1.2500,1.0125,0.3375,23.289114,11.977664,4.507570,5.953395,8


In [4]:
train.columns = ['id', 'sex', 'length', 'diameter', 'height', 'weight', 'shucked_weight', 'viscera_weight', 'shell_weight', 'age']
test.columns = ['id', 'sex', 'length', 'diameter', 'height', 'weight', 'shucked_weight', 'viscera_weight', 'shell_weight']

In [5]:
train.dtypes

id                  int64
sex                object
length            float64
diameter          float64
height            float64
weight            float64
shucked_weight    float64
viscera_weight    float64
shell_weight      float64
age                 int64
dtype: object

In [6]:
train.isnull().value_counts()

id     sex    length  diameter  height  weight  shucked_weight  viscera_weight  shell_weight  age  
False  False  False   False     False   False   False           False           False         False    74051
dtype: int64

In [7]:
train['sex'].unique()

array(['I', 'M', 'F'], dtype=object)

In [8]:
del train['id']
ids = test['id']
del test['id']
del train['sex']
del test['sex']

In [9]:
from sklearn.model_selection import train_test_split

y = train['age']
del train['age']
X = train

train_X, val_X, train_y, val_y = train_test_split(X, y)

In [10]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(train_X, train_y)

lr_pred = lr.predict(val_X)

In [11]:
from sklearn.linear_model import Ridge

ridge = Ridge()
ridge.fit(train_X, train_y)

ridge_pred = ridge.predict(val_X)

In [12]:
from xgboost import XGBRegressor

xgb_model = XGBRegressor(n_estimators = 1000, learning_rate = 1, random_state = 1)
xgb_model.fit(train_X, train_y, early_stopping_rounds = 5, eval_set = [(val_X, val_y)], verbose = False)

xgb_pred = xgb_model.predict(val_X)

/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


In [13]:
from catboost import CatBoostRegressor

cb_model = CatBoostRegressor(iterations = 200, random_state = 1, eval_metric = 'MAE')
cb_model.fit(train_X, train_y, eval_set = (val_X, val_y))

cb_pred = cb_model.predict(val_X)

Learning rate set to 0.255369
0:	learn: 2.0813546	test: 2.0575517	best: 2.0575517 (0)	total: 61.6ms	remaining: 12.2s
1:	learn: 1.8960378	test: 1.8697404	best: 1.8697404 (1)	total: 67.9ms	remaining: 6.72s
2:	learn: 1.7756180	test: 1.7492207	best: 1.7492207 (2)	total: 74.6ms	remaining: 4.9s
3:	learn: 1.6995799	test: 1.6721051	best: 1.6721051 (3)	total: 81.1ms	remaining: 3.97s
4:	learn: 1.6484148	test: 1.6225716	best: 1.6225716 (4)	total: 87.5ms	remaining: 3.41s
5:	learn: 1.6028901	test: 1.5790922	best: 1.5790922 (5)	total: 94.1ms	remaining: 3.04s
6:	learn: 1.5754920	test: 1.5532447	best: 1.5532447 (6)	total: 101ms	remaining: 2.78s
7:	learn: 1.5547147	test: 1.5331465	best: 1.5331465 (7)	total: 107ms	remaining: 2.56s
8:	learn: 1.5360738	test: 1.5166135	best: 1.5166135 (8)	total: 114ms	remaining: 2.41s
9:	learn: 1.5229780	test: 1.5041697	best: 1.5041697 (9)	total: 120ms	remaining: 2.28s
10:	learn: 1.5162116	test: 1.4972804	best: 1.4972804 (10)	total: 126ms	remaining: 2.16s
11:	learn: 1.5054

In [14]:
from sklearn.metrics import mean_absolute_error

lr_mae = mean_absolute_error(val_y, lr_pred)
ridge_mae = mean_absolute_error(val_y, ridge_pred)
xgb_mae = mean_absolute_error(val_y, xgb_pred)
cb_mae = mean_absolute_error(val_y, cb_pred)

best_mae = sorted([lr_mae, ridge_mae, xgb_mae, cb_mae])[0]

model = None
if best_mae == lr_mae:
    model = lr_model
elif best_mae == ridge_mae:
    model = ridge_model
else:
    model = xgb_model

pred = model.predict(test)

print("LR MAE:", lr_mae)
print("Ridge MAE:", ridge_mae)
print("XGB MAE:", xgb_mae)
print("CB MAE:", cb_mae)
print("Best MAE:", best_mae)

LR MAE: 1.5023624110221507
Ridge MAE: 1.5023807015983084
XGB MAE: 1.4563475969396276
CB MAE: 1.42467126625157
Best MAE: 1.42467126625157


In [15]:
output = pd.DataFrame({'id': ids, 'age': pred})
output.to_csv('submission.csv', index = False)
print('Your submission was successfully saved!')

Your submission was successfully saved!
